In [6]:
import os
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn import metrics
import xgboost as xgb
import matplotlib
import torch
import torch.utils.data as data_utils
import torch.nn as nn
import tensorflow as tf
import imblearn
from sklearn.model_selection import train_test_split
import collections
from sklearn.metrics import roc_curve, auc

In [4]:
date='16Mar2021'
dir_name='/preprocessing/200PU_2806'
workdir=os.getcwd()
files_dir='/data_cms_upgrade/hakimi'
os.makedirs(workdir+dir_name, exist_ok=True)
data_dir=workdir+dir_name
signal='elec'
bkg = 'QCD'
savedir = workdir + '/moo/'+ bkg + '/'
os.makedirs(savedir, exist_ok=True)

In [5]:
bkg

'QCD'

In [6]:
%%time
columns=['genpart_exeta',
         'genpart_pt',
 'cl3d_pt',
 'cl3d_eta',
 'cl3d_showerlength',
 'cl3d_coreshowerlength',
 'cl3d_firstlayer',
 'cl3d_maxlayer',
 'cl3d_seetot',
 'cl3d_seemax',
 'cl3d_spptot',
 'cl3d_sppmax',
 'cl3d_szz',
 'cl3d_srrtot',
 'cl3d_srrmax',
 'cl3d_srrmean',
 'cl3d_emaxe',
 'cl3d_hoe',
 'cl3d_meanz',
 'cl3d_layer10',
 'cl3d_layer50',
 'cl3d_layer90',
 'cl3d_ntc67',
 'cl3d_ntc90',
 'layer',
 'sample',
  'matches',
         
        ]


df_sig= pd.read_csv(data_dir+'/{}.csv'.format(signal), usecols=columns, low_memory=True)
if bkg == 'QCD':
    columns =['event', 'genjet_n', 'genjet_energy', 'genjet_pt', 'genjet_eta',
       'genjet_phi', 'cl3d_pt', 'cl3d_energy', 'cl3d_eta',
       'cl3d_phi', 
       'cl3d_showerlength', 'cl3d_coreshowerlength', 'cl3d_firstlayer',
       'cl3d_maxlayer', 'cl3d_seetot', 'cl3d_seemax', 'cl3d_spptot',
       'cl3d_sppmax', 'cl3d_szz', 'cl3d_srrtot', 'cl3d_srrmax', 'cl3d_srrmean',
       'cl3d_emaxe', 'cl3d_hoe', 'cl3d_meanz', 'cl3d_layer10', 'cl3d_layer50',
       'cl3d_layer90', 'cl3d_ntc67', 'cl3d_ntc90', 'layer', 'deta', 'dphi',
       'deltar', 'matches',]
elif bkg == 'PU':
    columns = ['event', 'cl3d_pt', 'cl3d_energy', 'cl3d_eta', 'cl3d_phi', 
       'cl3d_showerlength', 'cl3d_coreshowerlength', 'cl3d_firstlayer',
       'cl3d_maxlayer', 'cl3d_seetot', 'cl3d_seemax', 'cl3d_spptot',
       'cl3d_sppmax', 'cl3d_szz', 'cl3d_srrtot', 'cl3d_srrmax', 'cl3d_srrmean',
       'cl3d_emaxe', 'cl3d_hoe', 'cl3d_meanz', 'cl3d_layer10', 'cl3d_layer50',
       'cl3d_layer90', 'cl3d_ntc67', 'cl3d_ntc90', 'layer',]

df_bkg= pd.read_csv(data_dir+'/{}.csv'.format(bkg), usecols=columns, low_memory=True)
df_bkg['sample']=bkg

print('done')

done
CPU times: user 19.8 s, sys: 2.07 s, total: 21.8 s
Wall time: 21.9 s


In [7]:
"""df['genpart_exeta'][df['sample']=='PU']=0
df['genpart_pt'][df['sample']=='PU']=0
df['matches'][df['sample']=='PU']=0"""

"df['genpart_exeta'][df['sample']=='PU']=0\ndf['genpart_pt'][df['sample']=='PU']=0\ndf['matches'][df['sample']=='PU']=0"

# Config

In [8]:
genptcut=20
cl3dptcut=5
PU_cut = 20
etamin=1.6
etamax=2.9
#bkg='QCD' # 'pions' or 'PU'
standardize = "_unstandardized"
feature_set=16 # 1: baseline 4: minimal, 10-13 + new vars (12 -> PU 13-> pions)
opti=False
res_dir=workdir+'/res/res_pt{}_vs{}_featset{}{}'.format(genptcut, bkg, feature_set, standardize)
os.makedirs(res_dir, exist_ok=True)

In [9]:
from collections import Counter
count=Counter

In [10]:
%%time
df_sig['abseta']=np.abs(df_sig['cl3d_eta'])
df_bkg['abseta']=np.abs(df_bkg['cl3d_eta'])
sel = ((np.abs(df_sig['genpart_exeta'])>etamin) & (np.abs(df_sig['genpart_exeta'])<etamax )
       & (df_sig['genpart_pt']>genptcut)&  (df_sig['cl3d_pt']>cl3dptcut) 
       &(df_sig['abseta']>etamin) & (df_sig['abseta']<etamax ))
df_sig_cut=df_sig[sel]
df_sig_cut.dropna(inplace=True)
if bkg == 'QCD':
    sel = ((np.abs(df_bkg['cl3d_eta'])>etamin) & (np.abs(df_bkg['cl3d_eta'])<etamax )&(df_bkg['cl3d_pt']>cl3dptcut))
if bkg == 'pions':
    sel =  ((np.abs(df_bkg['genpart_exeta'])>etamin) & (np.abs(df_bkg['genpart_exeta'])<etamax )
       & (df_bkg['genpart_pt']>genptcut)&  (df_bkg['cl3d_pt']>cl3dptcut) 
       &(df_bkg['abseta']>etamin) & (df_bkg['abseta']<etamax ))
if bkg == 'PU':
    sel = ((np.abs(df_bkg['cl3d_eta'])>etamin) & (np.abs(df_bkg['cl3d_eta'])<etamax )&(df_bkg['cl3d_pt']>PU_cut))
df_bkg_cut=df_bkg[sel]
df_bkg_cut.dropna(inplace=True)
df_cut= df_sig_cut.append(df_bkg_cut)
print(count(df_cut['sample']))

/opt/exp_soft/llr/python/3.7.0/el7/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/opt/exp_soft/llr/python/3.7.0/el7/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/exp_soft/llr/python/3.7.0/el7/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


Counter({'elec': 207350, 'QCD': 135546})
CPU times: user 473 ms, sys: 138 ms, total: 611 ms
Wall time: 610 ms


In [11]:
%%time
#cut
algo_cut={}

def tolist(x):
    x.test=x.layer[1:-1].split(',')
    return x.test



#cut on cl3deta
df_cut['abseta']=np.abs(df_cut['cl3d_eta'])
sel=(df_cut['abseta']>etamin) & (df_cut['abseta']<etamax)
df_cut=df_cut[sel]
print('cl3d eta cut:', len(df_cut), len(df_cut[df_cut['sample']=='elec']))
#print('PU: ', len(df_cut[df_cut['sample']=='PU']))
#layer_pt preproc
df_cut['layer_pt']=df_cut.apply(tolist, axis=1)
df_cut.drop('layer', axis=1, inplace=True)
print("done")
def layering (x):
    return float(x.layer_pt[n])

##besoin de créer une variable par layer?
algo_layer={}
n_layers=len(df_cut['layer_pt'].iloc[0])
#print(n_layers)
layer_columns=[]

for n in range(n_layers):
    print('layering: {}/{}\r'.format(n+1,n_layers),end='', flush=True)
    df_cut['layer_{}'.format(n)]=df_cut.apply(layering, axis=1)
    layer_columns.append('layer_{}'.format(n))
print("done layering")



cl3d eta cut: 342896 207350
done
done layering36
CPU times: user 4min 26s, sys: 14.5 s, total: 4min 41s
Wall time: 4min 42s


In [12]:
Counter(df_cut['sample'])

Counter({'elec': 207350, 'QCD': 135546})

In [13]:
# build new vars

#abseta
#df_cut['abseta']=np.abs(df_cut['cl3d_eta'])

#variance
df_cut['varee']=df_cut['cl3d_seetot']**2
df_cut['varpp']=df_cut['cl3d_spptot']**2
df_cut['varzz']=df_cut['cl3d_szz']**2
df_cut['varrr']=df_cut['cl3d_srrtot']**2

#EoT: pt(Ecal)/pt(Tot)
nLayerEcal= 14
sumE=0

sumT=0
for i in range(n_layers):
    #print('Totlayer_{}'.format(i))
    sumT+=df_cut['layer_{}'.format(i)]
    
    
for i in range(1,nLayerEcal+1):
    #print('Ecal layer_{}'.format(i))
    sumE+=df_cut['layer_{}'.format(i)]
df_cut['EoT']=sumE/sumT

#fraction of pt in first x  layers and last x layers
maxfirst = 5
maxlast=10

for n in range(1,maxfirst+1):
    Sum=0
    for i in range(1,n+1):
        Sum+=df_cut['layer_{}'.format(i)]
    df_cut['first_{}'.format(n)]= Sum/sumT
    
for n in range(1,maxlast):
    Sum=0
    #print(n, ' layers')
    for i in range(n_layers - n, n_layers):
        #print(n,i)
        Sum+=df_cut['layer_{}'.format(i)]
    #print('last_{}'.format(n), sum/sumT)
    df_cut['last_{}'.format(n)]= Sum/sumT
    
    

for n in range(1,maxfirst+1):
    Sum=0
    for i in range(1,n+1):
        #print('layer_{}'.format(nLayerEcal+i))
        Sum+=df_cut['layer_{}'.format(nLayerEcal+i)]
    df_cut['firstHcal_{}'.format(n)]= Sum/sumT
    
    
# Emaxx : pt in x layers around Elec max layer (5)
maxpos=5

#print('emax_1')
Sum=0
Sum+=df_cut['layer_{}'.format(maxpos)]
df_cut['Emax_1']=Sum/sumT
 
#print('emax_2L')
Sum=0
for i in range(maxpos-1, maxpos+1):
    #print(i)
    Sum+=df_cut['layer_{}'.format(i)]
    df_cut['Emax_2L']=Sum/sumT
    
#print('emax_2R')
Sum=0
for i in range(maxpos, maxpos+2):
    #print(i)
    Sum+=df_cut['layer_{}'.format(i)]
    df_cut['Emax_2R']=Sum/sumT
    
#print('emax_3')
Sum=0
for i in range(maxpos-1, maxpos+2):
    #print(i)
    Sum+=df_cut['layer_{}'.format(i)]
    df_cut['Emax_3']=Sum/sumT
#print('emax_4G')
Sum=0
for i in range(maxpos-2, maxpos+2):
    #print(i)
    Sum+=df_cut['layer_{}'.format(i)]
    df_cut['Emax_4L']=Sum/sumT
#print('emax_4R')
Sum=0
for i in range(maxpos-1, maxpos+3):
    #print(i)
    Sum+=df_cut['layer_{}'.format(i)]
    df_cut['Emax_4R']=Sum/sumT
#print('emax_5')
Sum=0
for i in range(maxpos-2, maxpos+3):
    #print(i)
    Sum+=df_cut['layer_{}'.format(i)]
    df_cut['Emax_5']=Sum/sumT
    
    
    

In [14]:
%%time
#do bitmaps with group
def ebm0(x):
    ebm=[]
    thr=0
    for i in range(1,nLayerEcal+1):
        #print(float(x.layer_pt[i])>thr)
        ebm.append(int(float(x.layer_pt[i])>thr)) 
    #print(ebm)
    return(np.array(ebm).dot(2**np.arange(len(ebm))[::-1]))
def ebm1(x):
    ebm=[]
    thr=1
    for i in range(1,nLayerEcal+1):
        #print(float(x.layer_pt[i])>thr)
        ebm.append(int(float(x.layer_pt[i])>thr)) 
    #print(ebm)
    return(np.array(ebm).dot(2**np.arange(len(ebm))[::-1]))
def hbm(x):
    ebm=[]
    thr=0
    for i in range(nLayerEcal+1,n_layers):
        #print(float(x.layer_pt[i])>thr)
        ebm.append(int(float(x.layer_pt[i])>thr)) 
    #print(ebm)
    return(np.array(ebm).dot(2**np.arange(len(ebm))[::-1]))
        
df_cut['ebm0']=df_cut.apply(ebm0, axis=1)
df_cut['ebm1']=df_cut.apply(ebm1, axis=1)
df_cut['hbm']=df_cut.apply(hbm, axis=1)

CPU times: user 4min 13s, sys: 7.95 s, total: 4min 21s
Wall time: 4min 15s


In [19]:
ebm= [1,0,1,0,1,0,1,0,1,0,1,0,1,0,1]
np.array(ebm).dot(2**np.arange(len(ebm))[::-1])

21845

In [13]:
# try in last lay -> first

In [14]:
%%time
#do bitmaps with group
def reverse_ebm0(x):
    ebm=[]
    thr=0
    for i in range(nLayerEcal, 0, -1):
        #print(float(x.layer_pt[i])>thr)
        ebm.append(int(float(x.layer_pt[i])>thr)) 
    #print(ebm)
    return(np.array(ebm).dot(2**np.arange(len(ebm))[::-1]))
def reverse_ebm1(x):
    ebm=[]
    thr=1
    for i in range(nLayerEcal, 0, -1):
        #print(float(x.layer_pt[i])>thr)
        ebm.append(int(float(x.layer_pt[i])>thr)) 
    #print(ebm)
    return(np.array(ebm).dot(2**np.arange(len(ebm))[::-1]))
def reverse_hbm(x):
    ebm=[]
    thr=0
    for i in range(n_layers-1,nLayerEcal-1, -1):
        #print(float(x.layer_pt[i])>thr)
        ebm.append(int(float(x.layer_pt[i])>thr)) 
    #print(ebm)
    return(np.array(ebm).dot(2**np.arange(len(ebm))[::-1]))
        
df_cut['reverse_ebm0']=df_cut.apply(reverse_ebm0, axis=1)
df_cut['reverse_ebm1']=df_cut.apply(reverse_ebm1, axis=1)
df_cut['reverse_hbm']=df_cut.apply(reverse_hbm, axis=1)
print('done')

done
CPU times: user 4min 23s, sys: 8.76 s, total: 4min 32s
Wall time: 4min 25s


In [15]:
#bitmap: for quantization, group layers
# max 16 bits (in realityn, 14 layers for ebm -> 14bits)
# possibilities: 7 bits (/2), 5 bist(/3), 4 bits(/4), 3bits(/5), 2 bits (/7)
x=df_cut.iloc[0]    
ebm=[]
thr=0

for i in range(1,nLayerEcal+1):
    #print(float(x.layer_pt[i])>thr)
    ebm.append(int(float(x['layer_pt'][i])>thr)) 
#print(ebm)

In [16]:
np.random.choice(a=[0, 1], size=14)  

array([1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0])

In [17]:
print(df_cut.columns.tolist())

['abseta', 'cl3d_coreshowerlength', 'cl3d_emaxe', 'cl3d_energy', 'cl3d_eta', 'cl3d_firstlayer', 'cl3d_hoe', 'cl3d_layer10', 'cl3d_layer50', 'cl3d_layer90', 'cl3d_maxlayer', 'cl3d_meanz', 'cl3d_ntc67', 'cl3d_ntc90', 'cl3d_phi', 'cl3d_pt', 'cl3d_seemax', 'cl3d_seetot', 'cl3d_showerlength', 'cl3d_sppmax', 'cl3d_spptot', 'cl3d_srrmax', 'cl3d_srrmean', 'cl3d_srrtot', 'cl3d_szz', 'deltar', 'deta', 'dphi', 'event', 'genjet_energy', 'genjet_eta', 'genjet_n', 'genjet_phi', 'genjet_pt', 'genpart_exeta', 'genpart_pt', 'matches', 'sample', 'layer_pt', 'layer_0', 'layer_1', 'layer_2', 'layer_3', 'layer_4', 'layer_5', 'layer_6', 'layer_7', 'layer_8', 'layer_9', 'layer_10', 'layer_11', 'layer_12', 'layer_13', 'layer_14', 'layer_15', 'layer_16', 'layer_17', 'layer_18', 'layer_19', 'layer_20', 'layer_21', 'layer_22', 'layer_23', 'layer_24', 'layer_25', 'layer_26', 'layer_27', 'layer_28', 'layer_29', 'layer_30', 'layer_31', 'layer_32', 'layer_33', 'layer_34', 'layer_35', 'varee', 'varpp', 'varzz', 'varr

In [18]:


if bkg == 'pions':
    name = 'minimal w/ var & best pions '
    features =['abseta', 'cl3d_firstlayer', 'varee', 'varpp', 'varzz', 'varrr',
                 'cl3d_meanz', 'cl3d_showerlength','cl3d_coreshowerlength','EoT',
                'Emax_5', 'ebm1', 'Emax_4R', 'hbm', 'firstHcal_1', 'reverse_ebm1']
    
        
elif bkg == 'PU':
    name = 'minimal w/ var & best PU '
    features = ['abseta', 'cl3d_firstlayer', 'varee', 'varpp', 'varzz', 'varrr',
                 'cl3d_meanz', 'cl3d_showerlength','cl3d_coreshowerlength', 'EoT',
                'firstHcal_5', 'Emax_4R', 'ebm1', 'Emax_2R', 'firstHcal_1', 'Emax_5']
    
elif bkg == 'QCD':
    name = 'minimal w/ var & best QCD '
    features=  ['abseta', 'cl3d_firstlayer', 'varee', 'varpp', 'varzz', 'varrr',
                 'cl3d_meanz', 'cl3d_showerlength','cl3d_coreshowerlength', 'EoT','ebm1', 'Emax_4R',
                'reverse_ebm1', 'first_2', 'Emax_2R', 'Emax_5', 
                ]
    

In [19]:
#chose signal and background
#weight calc

df_cut['signal']= 5
df_cut['weight']=0
df_cut['signal'][df_cut['sample']=='elec']= 1
df_cut['signal'][df_cut['sample']!='elec'] = 0

if bkg == 'pions':
    print('pions')

    data=df_cut[df_cut['sample']!='PU']
    tot_event=len(data)
    sig_event=len(data[data['sample']=='elec'])
    bkg_event=len(data[data['sample']=='pions'])
    ws=tot_event/sig_event
    wb=tot_event/bkg_event
    #print(ws)
    data['weight'][data['signal']==0]=wb
    #print(np.unique(data['weight']))

    #print(wb)
    data['weight'][data['signal']==1]=ws
    #print(np.unique(data['weight']))
elif bkg == 'QCD':
    print('QCD')

    data=df_cut
    tot_event=len(data)
    sig_event=len(data[data['sample']=='elec'])
    bkg_event=len(data[data['sample']=='QCD'])
    ws=tot_event/sig_event
    wb=tot_event/bkg_event
    #print(ws)
    data['weight'][data['signal']==0]=wb
    #print(np.unique(data['weight']))

    #print(wb)
    data['weight'][data['signal']==1]=ws
    #print(np.unique(data['weight']))

elif bkg == 'PU':
    print('PU')
    data=df_cut[df_cut['sample']!='pion']
    tot_event=len(data)
    sig_event=len(data[data['sample']=='elec'])
    bkg_event=len(data[data['sample']=='PU'])
    ws=tot_event/sig_event
    wb=tot_event/bkg_event

    data['weight'][data['signal']==0]=wb
    data['weight'][data['signal']==1]=ws
print('signal weight = {}, bkg weight ={}'.format(ws, wb))

/opt/exp_soft/llr/python/3.7.0/el7/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/exp_soft/llr/python/3.7.0/el7/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


QCD
signal weight = 1.6537062937062936, bkg weight =2.5297389816003424


/opt/exp_soft/llr/python/3.7.0/el7/lib/python3.7/site-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [20]:
import matplotlib
font = {'family' : 'normal',
        'weight' : 'normal',
        'size'   : 18}

matplotlib.rc('font', **font)

In [21]:
balancing='comb' #'SMOTE', 'weights', 'ada', 'under', 'comb'

In [22]:
Counter= collections.Counter

In [23]:
print(data.columns.to_list())

['abseta', 'cl3d_coreshowerlength', 'cl3d_emaxe', 'cl3d_energy', 'cl3d_eta', 'cl3d_firstlayer', 'cl3d_hoe', 'cl3d_layer10', 'cl3d_layer50', 'cl3d_layer90', 'cl3d_maxlayer', 'cl3d_meanz', 'cl3d_ntc67', 'cl3d_ntc90', 'cl3d_phi', 'cl3d_pt', 'cl3d_seemax', 'cl3d_seetot', 'cl3d_showerlength', 'cl3d_sppmax', 'cl3d_spptot', 'cl3d_srrmax', 'cl3d_srrmean', 'cl3d_srrtot', 'cl3d_szz', 'deltar', 'deta', 'dphi', 'event', 'genjet_energy', 'genjet_eta', 'genjet_n', 'genjet_phi', 'genjet_pt', 'genpart_exeta', 'genpart_pt', 'matches', 'sample', 'layer_pt', 'layer_0', 'layer_1', 'layer_2', 'layer_3', 'layer_4', 'layer_5', 'layer_6', 'layer_7', 'layer_8', 'layer_9', 'layer_10', 'layer_11', 'layer_12', 'layer_13', 'layer_14', 'layer_15', 'layer_16', 'layer_17', 'layer_18', 'layer_19', 'layer_20', 'layer_21', 'layer_22', 'layer_23', 'layer_24', 'layer_25', 'layer_26', 'layer_27', 'layer_28', 'layer_29', 'layer_30', 'layer_31', 'layer_32', 'layer_33', 'layer_34', 'layer_35', 'varee', 'varpp', 'varzz', 'varr

# Multi objective optimization

## 1. Select genes
Here the genesa re the different paramaters of the optimization, so the quantization paramaters for each feature (add 0 parameter for forgetting the feature?

-> alleles for each gene is the number of bits [2-12], and for now method =qauntile
-> add exception for discrete variables: 2**nbits <= range

In [24]:
def maxbits(feat, maxbit):
    if feat.dtype == 'float64':
        return maxbit
    elif feat.dtype =='int64':
        nval = len(np.unique(feat))
        print(nval)
        for i in range(1,maxbit) : 
            if 2**i >= nval: return(i)
        return(i)

In [25]:
maxbits(data['ebm1'],16)

3695


12

In [26]:
#change discrete functuions type to int
discrete_cols = ['cl3d_showerlength','cl3d_coreshowerlength', 'cl3d_firstlayer', 'cl3d_maxlayer' ]
data[discrete_cols]=data[discrete_cols].astype('int64')


In [27]:

X_train, X_test, y_train, y_test = train_test_split(data[features], 
                                                    data[['signal','weight','cl3d_eta','cl3d_pt', 'genpart_pt', 'genpart_exeta']],
                                                    test_size=0.2,random_state=42)
train_untouched=xgb.DMatrix(data=X_train,label=y_train['signal'],feature_names=features)
y_train_untouched=y_train
X_train_untouched=X_train
over_ratio = 0.2
under_ratio = 0.5
if bkg == 'QCD':
    over_ratio = 0.8
    under_ratio = 1
over=imblearn.over_sampling.SMOTE(sampling_strategy=over_ratio, random_state=42)
under= imblearn.under_sampling.RandomUnderSampler(sampling_strategy=under_ratio, random_state=42)
X_train, y_train=over.fit_resample(X_train,y_train['signal'])
X_train, y_train=under.fit_resample(X_train, y_train)
y_train=pd.DataFrame(y_train)
y_train['weight']=1



In [28]:
import pymoo
#problem definition

In [29]:
#MOO inputs: nvar [0-12], max boost rounds[10-500], max_depth[2-8], eta [1-4] (negztive power of 10)

#nvar
nvar = len (data[features].columns) + 3
print('nvar',nvar)

#nobj
nobj = 3 # efficiency and number of bits used + model complexity

#nconstr
nconstr = 0

bitmin= 0 #0 if we want to remove features

# varaiables lower bound xl
#features number of bits
xl=[bitmin for feat in data[features].columns]
#add bdt params
xl.extend([10,2,1])
print('lower bounds',xl)


bitmax = 12
#upper bound
#features number of bits
xu =[maxbits(data[feat], bitmax) for feat in data[features].columns]
#add bdt params
xu.extend([500,8,5])
print ('upper bounds',xu)

param = {
        'nthread' : 8,
        # Parameters that we are going to tune.
        'max_depth':2,
        'eta':0.1,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        # Other parameters
        'objective':'binary:logistic',
        'eval_metric': 'auc', 
        'alpha':0.01,
        'lambda':10,
    }

nvar 19
lower bounds [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10, 2, 1]
22
50
33
3695
3695
upper bounds [12, 5, 12, 12, 12, 12, 12, 6, 6, 12, 11, 12, 11, 12, 12, 12, 500, 8, 5]


In [7]:
np.sum([12, 5, 12, 12, 12, 12, 12, 6, 6, 12, 11, 12, 11, 12, 12, 12,])

171

In [30]:
# quantization

def quantize(feat, nbits, method, fmin, fmax): #feat is the feature to quantize, nbist the number of bits, method: 'uniform' or 'percentile'
    nbins= 2**nbits
    if feat.dtype == 'float64':
        method = method
    elif feat.dtype == 'int64':
        method = 'uniform'
    if method == 'uniform':
        bins=np.linspace(fmin, fmax, nbins+1)
    elif method == 'percentile':
        bins=[np.percentile(feat, n) for n in np.linspace(0,100,nbins+1)]
    else :
        print('Error: Invalid method')
        return
    
    
    #print(feat.dtype,nbits, method)
    return np.digitize(feat, bins)

def quantize_ebm0(x,nb):
    qbm=[]
    nlay=14
    fuse_lay=int(np.ceil(nlay/nb))
    print(" nb", nb)
    bm = int(x.ebm0)
    print('bm =', bm)
    bm = list(bin(bm))[2:]
    print('bits =', bm)
    while len(bm) < nlay:
        bm.insert(0,0)
    print('bits(nlay)=', bm)
    for j in range(0,14,fuse_lay):
        val=0
        #print("j",j)
        for k in range(fuse_lay):
            #print("k",k)
            if j+k <=13:
                val +=int(bm[j+k])
        #print("val", val)
        qbm.append(int(val>0))
    print(qbm)
    return qbm


def quantize_ebm1(x,nb):
    qbm=[]
    nlay=14
    fuse_lay=int(np.ceil(nlay/nb))
    print(" nb", nb)
    bm = int(x.ebm1)
    print('bm =', bm)
    bm = list(bin(bm))[2:]
    print('bits =', bm)
    while len(bm) < nlay:
        bm.insert(0,0)
    print('bits(nlay)=', bm)
    for j in range(0,14,fuse_lay):
        val=0
        #print("j",j)
        for k in range(fuse_lay):
            #print("k",k)
            if j+k <=13:
                val +=int(bm[j+k])
        #print("val", val)
        qbm.append(int(val>0))
    print(qbm)
    return qbm
def quantize_hbm(x,nb):
    qbm=[]
    nlay=21
    fuse_lay=int(np.ceil(nlay/nb))
    print(" nb", nb)
    bm = int(x.hbm)
    print('bm =', bm)
    bm = list(bin(bm))[2:]
    print('bits =', bm)
    while len(bm) < nlay:
        bm.insert(0,0)
    print('bits(nlay)=', bm)
    for j in range(0,nlay,fuse_lay):
        val=0
        #print("j",j)
        for k in range(fuse_lay):
            #print("k",k)
            if j+k <=13:
                val +=int(bm[j+k])
        #print("val", val)
        qbm.append(int(val>0))
    print(qbm)
    return qbm
def quantize_reverse_ebm0(x,nb):
    qbm=[]
    nlay=14
    fuse_lay=int(np.ceil(nlay/nb))
    print(" nb", nb)
    bm = int(x.reverese_ebm0)
    print('bm =', bm)
    bm = list(bin(bm))[2:]
    print('bits =', bm)
    while len(bm) < nlay:
        bm.insert(0,0)
    print('bits(nlay)=', bm)
    for j in range(0,14,fuse_lay):
        val=0
        #print("j",j)
        for k in range(fuse_lay):
            #print("k",k)
            if j+k <=13:
                val +=int(bm[j+k])
        #print("val", val)
        qbm.append(int(val>0))
    print(qbm)
    return qbm
def quantize_reverse_ebm1(x,nb):
    qbm=[]
    nlay=14
    fuse_lay=int(np.ceil(nlay/nb))
    print(" nb", nb)
    bm =int(x.reverese_ebm1)
    print('bm =', bm)
    bm = list(bin(bm))[2:]
    print('bits =', bm)
    while len(bm) < nlay:
        bm.insert(0,0)
    print('bits(nlay)=', bm)
    for j in range(0,14,fuse_lay):
        val=0
        #print("j",j)
        for k in range(fuse_lay):
            #print("k",k)
            if j+k <=13:
                val +=int(bm[j+k])
        #print("val", val)
        qbm.append(int(val>0))
    print(qbm)
    return qbm
def quantize_reverese_hbm(x,nb):
    qbm=[]
    nlay=21
    fuse_lay=int(np.ceil(nlay/nb))
    print(" nb", nb)
    bm = int(x.reverse_hbm)
    print('bm =', bm)
    bm = list(bin(bm))[2:]
    print('bits =', bm)
    while len(bm) < nlay:
        bm.insert(0,0)
    print('bits(nlay)=', bm)
    for j in range(0,nlay,fuse_lay):
        val=0
        #print("j",j)
        for k in range(fuse_lay):
            #print("k",k)
            if j+k <=13:
                val +=int(bm[j+k])
        #print("val", val)
        qbm.append(int(val>0))
    print(qbm)
    return qbm

In [31]:
df_cut.head(1)['hbm']

4    1310720
Name: hbm, dtype: int64

In [32]:
nlay=14
nb=13

print(nlay/int(np.ceil(nlay/nb)))

7.0


In [33]:
param

{'nthread': 8,
 'max_depth': 2,
 'eta': 0.1,
 'subsample': 0.8,
 'colsample_bytree': 0.8,
 'objective': 'binary:logistic',
 'eval_metric': 'auc',
 'alpha': 0.01,
 'lambda': 10}

In [2]:
xu

NameError: name 'xu' is not defined

In [35]:
len (data[features].columns)

16

In [36]:
eta=float

In [37]:
def train_quantized(x):
    nfeat=len (data[features].columns)
    #x[16]=boost rounds, x[17]=max depth,  x[21]=eta
    param['eta']= float(10)**((-1)*x[nfeat+2])
    param['max_depth']=x[nfeat+1]
    #param['alpha']=x[nfeat+3]
    #param['lambda']=x[nfeat+4]
    alpha = 1
    num_boost_round=x[nfeat]
    #print('test : boost rounds {} max_depth {} eta {}'.format(num_boost_round, param['max_depth'], param['eta']))
    
    qtrain=pd.DataFrame()
    qtest=pd.DataFrame()
    feat_kept=features.copy()
    tot_bits= 0
    for i,feat in enumerate(X_train.columns):
        #print("feature :", feat, "quantized on {} bits".format(x[i]))
        if x[i]==0: #drop feat
            #print("dropping {} from {}".format(feat, feat_kept))
            feat_kept.remove(feat)
        elif 'bm' not in 'feat':
            fmin, fmax= np.percentile(X_train[feat], 1),np.percentile(X_train[feat], 99)
            qtrain[feat]=quantize(X_train[feat], x[i], 'uniform', fmin, fmax)
            qtest[feat]=quantize(X_test[feat], x[i], 'uniform', fmin, fmax)
            tot_bits += x[i]
        elif feat == 'ebm0':
            qtrain[feat]=X_train.apply(quantize_ebm0, nb = x[i], axis=1)
            qtest[feat]=X_test.apply(quantize_ebm0, nb = x[i], axis=1)
            nlay=14
            bits=nlay/int(np.ceil(nlay/x[i]))
            if bits > 0:
                tot_bits += bits
        elif feat == 'ebm1':
            qtrain[feat]=X_train.apply(quantize_ebm1, nb = x[i], axis=1)
            qtest[feat]=X_test.apply(quantize_ebm1, nb = x[i], axis=1)
            nlay=14
            bits=nlay/int(np.ceil(nlay/x[i]))
            if bits > 0:
                tot_bits += bits
        elif feat == 'hbm':
            qtrain[feat]=X_train.apply(quantize_hbm, nb = x[i], axis=1)
            qtest[feat]=X_test.apply(quantize_hbm, nb = x[i], axis=1)
            nlay=21
            bits=nlay/int(np.ceil(nlay/x[i]))
            tot_bits += nlay/int(np.ceil(nlay/x[i]))
        elif feat == 'reverse_ebm0':
            qtrain[feat]=X_train.apply(quantize_reverse_ebm0, nb = x[i], axis=1)
            qtest[feat]=X_test.apply(quantize_reverse_ebm0, nb = x[i], axis=1)
            nlay=14
            bits=nlay/int(np.ceil(nlay/x[i]))
            if bits > 0:
                tot_bits += bits
        elif feat == 'reverse_ebm1':
            qtrain[feat]=X_train.apply(quantize_reverse_ebm1, nb = x[i], axis=1)
            qtest[feat]=X_test.apply(quantize_reverse_ebm1, nb = x[i], axis=1)
            nlay=14
            bits=nlay/int(np.ceil(nlay/x[i]))
            if bits > 0:
                tot_bits += bits
        elif feat == 'reverse_hbm':
            qtrain[feat]=X_train.apply(quantize_reverse_hbm, nb = x[i], axis=1)
            qtest[feat]=X_test.apply(quantize_reverse_hbm, nb = x[i], axis=1)
            nlay=21
            bits=nlay/int(np.ceil(nlay/x[i]))
            if bits > 0:
                tot_bits += bits
    #print("test")       
    #print(qtrain.head(1))
    train= xgb.DMatrix(data=qtrain,label=y_train['signal'], feature_names=feat_kept, weight=y_train['weight'])
    test= xgb.DMatrix(data=qtest,label=y_test['signal'],feature_names=feat_kept)
    #print("test2")
    #use gpu if available
    gpu=tf.test.is_gpu_available(
        cuda_only=False, min_cuda_compute_capability=None
    )
    if gpu==True:
        param['tree_method']='gpu_hist'

    """ #num_boost_round=200
    es_rounds=3

    metrics={'logloss','auc'} #last one used for es
    # set up cross validation:
    #print('beginning cv')
    cv_results = xgb.cv(
        param,
        train,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics=metrics,
        #feval=SoverB,
        early_stopping_rounds=es_rounds,
        shuffle=True,
        verbose_eval=None
    )

    ## we define best score as the best test_auc penalized by the gap between train and test auc
    #alpha=1 #(penalization)

    best_score=(cv_results['test-auc-mean']-alpha*abs(cv_results['test-auc-mean']-cv_results['train-auc-mean'])).max()
    boost_rounds = (cv_results['test-auc-mean']-alpha*abs(cv_results['test-auc-mean']-cv_results['train-auc-mean'])).argmax()
    """#Now train and fit best parameters BDT
    eval_result = {}
    BDT = xgb.train(
        param,
        train,
        num_boost_round=num_boost_round,
        #feval = SoverB,
        evals=[(train, 'Train'),(test, "Test")],
        evals_result = eval_result,
        verbose_eval=None,
    )
    pred= BDT.predict(test)


    fpr, tpr, threshold = roc_curve(y_test['signal'],pred, pos_label=1)
    fpr.sort()
    tpr.sort()
    roc_test=pd.DataFrame({'tpr':tpr,'fpr':fpr, 'threshold':threshold})

    #return bkg eff at 0.99 signal efficiency
    #f1=np.interp(0.99,roc_test['tpr'],roc_test['fpr'])


    #return auc at > cut sig eff
    cut=0.8

    roc_thr=roc_test[roc_test['tpr']>0.8]
    if len(roc_thr)>3:
        f1= (-1)*auc(roc_thr['fpr'], roc_thr['tpr']) # negative to be a minimization
    else : 
        f1 = 0
    #f2 = total number of bits used
    f2=tot_bits
    #f3 = model complexityk, aka nb of splits
    f3 = len(BDT.trees_to_dataframe()[BDT.trees_to_dataframe()['Feature']!='Leaf'])
    print('parameters: {} \n bkg efficiency = {}, using {} bits and {} splits'.format(x,f1, f2, f3))
    return ([f1, f2, f3], pred, BDT, qtrain, feat_kept)

In [38]:
def plot_shap(BDT, data, feats):
    import shap
    # select a set of background examples to take an expectation over
    #background = X_train.to_numpy()[np.random.choice(X_train.shape[0], 1000, replace=False)]

    # explain predictions of the model on four images
    explainer = shap.Explainer(BDT)
    a= qtrain[feat_kept][(y_train['signal']==1).values].sample(2000)
    b = qtrain[feat_kept][(y_train['signal']==0).values].sample(2000)
    shap_df= pd.concat([a,b])

    #shap_values = explainer.shap_values(X_test.to_numpy()[np.random.choice(X_test.shape[0], 1000, replace=False)])
    shap_values = explainer.shap_values(shap_df.to_numpy())
    col_names=[val for val in shap_df.columns if val != 'abseta']
    col_names.append('abseta')

    plt.figure(figsize=(10,5))
    #shap.summary_plot(shap_values, X_test.to_numpy()[np.random.choice(X_test.shape[0], 1000, replace=False)], 
    #                  features,max_display=99, show=False, alpha = 0.7 )

    shap.summary_plot(shap_values, shap_df.to_numpy() ,shap_df.columns,max_display=99, show=False, alpha = 0.8 )
    plt.xlabel('Shap value', fontsize=16)

EoT 1 : 99.8468
EoT 10:99.8507
EoT 0:99.8436

#x[16]=boost rounds, x[17]=max depth, x[18]= auc penal, x[19]= alpha, x[20]=lambda, x[21]=eta
x=[  3.        ,   0.        ,   1.        ,   5.        ,
         3.        ,   9.        ,   0.        ,   1.        ,
         0.        ,   10.        ,  10.        ,   4.        ,
         3.        ,   0.        ,   1.        ,   0.     ,30 ,4,1,0.01,10,0.1   ]
out, pred, BDT, qtrain, feat_kept=train_quantized(x) 
print(out)
plot_shap(BDT, qtrain, feat_kept)
#plt.savefig(res_dir+'/importance_violin.png', bbox_inches='tight')


In [39]:
import numpy as np

from pymoo.factory import get_algorithm, get_crossover, get_mutation, get_sampling, get_selection
from pymoo.optimize import minimize
from pymoo.model.problem import Problem


class MyProblem(Problem):

    def __init__(self):
        super().__init__(n_var=nvar, n_obj=nobj, n_constr=nconstr, xl=xl, xu=xu, type_var=int, elementwise_evaluation=True)

    def _evaluate(self, x, out, *args, **kwargs):
        #define evaluations functions (minimization)
        #f1 = bkg eff at 99% signal efficiency
        #print(x)
        #it=pd.DataFrame([x], columns=features)
        #print(it)
        
        out["F"] = train_quantized(x)[0]
problem=MyProblem()

In [40]:
import random


def initial_pop(pop_size=16, rand= True):
    #initializatio
    #x[0-15] = nbits, x[16]=boost rounds, x[17]=max depth, x[18]=eta
    nfeat=len (data[features].columns)
    max_val=12
    pop_size=pop_size
    sampling=[]
    #hyperdiag
    for i in range(1,max_val+1):
        current=[i for j in range(nfeat)]
        #BDT params: min + i*range/12
        current.append(round(10+(i-1)*(490/(max_val-1)))) # boost rounds
        current.append(round(2+(i-1)*(6/(max_val-1)))) # depth
        current.append(round(1+(i-1)*(3/(max_val-1)))) # eta
        sampling.append(current)
    #add random pop until pop_size is reached
    if rand:
        for i in range(pop_size-max_val):
            current=[random.randint(0,12) for j in range(nfeat)]
            current.append(random.randint(10,500))
            current.append(random.randint(2,8))
            current.append(random.randint(1,3))
            sampling.append(current)
        sampling=np.array(sampling)
    return(sampling)

NameError: name 'ul' is not defined

In [41]:
pd.options.mode.chained_assignment = None  # default='warn'    
import warnings
warnings.filterwarnings("ignore", category=FutureWarning) 
warnings.filterwarnings("ignore", category=DeprecationWarning)


#hyperdiagonal sampling:
#range=[0,12] (add if 13 -> no quantization?but then, how to quantify number of bits used?)
max_val=12
pop_size=100
sampling=initial_pop(pop_size, True)



method = get_algorithm("nsga2",
                       pop_size=pop_size,
                       sampling=sampling,
                       crossover=get_crossover("int_sbx", prob=1.0, eta=3.0), #def 1,3
                       mutation=get_mutation("int_pm", eta=3.0), # def 3
                       eliminate_duplicates=True,
                       )


res = minimize(problem,
               method,
               termination=('n_gen', 60),
               seed=42,
               save_history=True,
               verbose=True
               )

print("Best solution found: %s" % res.X)
print("Function value: %s" % res.F)
print("Constraint violation: %s" % res.CV)

W0108 16:04:37.002239 139784885139264 deprecation.py:323] From <ipython-input-37-63e70182b9eb>:74: is_gpu_available (from tensorflow.python.framework.test_util) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


parameters: [ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 10  2  1] 
 bkg efficiency = -0.9547564096440782, using 16 bits and 30 splits
parameters: [ 2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2 55  3  1] 
 bkg efficiency = -0.9942070595548059, using 32 bits and 356 splits
parameters: [ 3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3 99  3  2] 
 bkg efficiency = -0.9948009279513554, using 48 bits and 688 splits
parameters: [  4   4   4   4   4   4   4   4   4   4   4   4   4   4   4   4 144   4
   2] 
 bkg efficiency = -0.9963872416762044, using 64 bits and 2104 splits
parameters: [  5   5   5   5   5   5   5   5   5   5   5   5   5   5   5   5 188   4
   2] 
 bkg efficiency = -0.9973862399761391, using 80 bits and 2771 splits
parameters: [  6   6   6   6   6   6   6   6   6   6   6   6   6   6   6   6 233   5
   2] 
 bkg efficiency = -0.9980817604548327, using 96 bits and 6581 splits
parameters: [  7   7   7   7   7   7   7   7   7   7   7   7   7   7   7   7 277   5
   3] 
 bkg e

parameters: [  8   3   6  10   5   0   9   9   3   0   7   8  12   0  12   2 459   5
   2] 
 bkg efficiency = -0.9972216157063133, using 94 bits and 13424 splits
parameters: [  4  11   6   7   5   3   9   2  11   3   1  10   8  10   0   2 492   8
   3] 
 bkg efficiency = -0.9976444243875235, using 92 bits and 60113 splits
parameters: [  1   7  12   9   0   7   3   5   6   6   1   1   6  10   4  11 347   6
   2] 
 bkg efficiency = -0.9982215884936743, using 89 bits and 18565 splits
parameters: [  6  12   7   8  12   4   8  10   1   7   7   2   8   2   2   1 150   2
   2] 
 bkg efficiency = -0.996198021504011, using 97 bits and 450 splits
parameters: [  2   8   2   6   3   1   3   1  10   1   8   7   0  11  11   6 268   7
   2] 
 bkg efficiency = -0.996151056908317, using 80 bits and 23642 splits
parameters: [  6   1   5   3   0   2   3  11   1   6  10   3   9   9   3  12 170   2
   1] 
 bkg efficiency = -0.9980737593298868, using 84 bits and 506 splits
parameters: [ 4  8  0  8  7 12  9 

/home/llr/cms/hakimi/.local/lib/python3.7/site-packages/pymoo/operators/crossover/simulated_binary_crossover.py:40: RuntimeWarning: divide by zero encountered in power
  alpha = 2.0 - np.power(beta, -(self.eta + 1.0))
/home/llr/cms/hakimi/.local/lib/python3.7/site-packages/pymoo/operators/crossover/simulated_binary_crossover.py:45: RuntimeWarning: invalid value encountered in power
  betaq[mask] = np.power((rand * alpha), (1.0 / (self.eta + 1.0)))[mask]


parameters: [  0   3   8   8  10   1   6   6   2   4   3  12   9   3   9   1 130   3
   3] 
 bkg efficiency = -0.9906363822602068, using 85 bits and 908 splits
parameters: [  2   6   6   3   1  11  12   5   6   4  11   0   1   9   6   5 489   5
   2] 
 bkg efficiency = -0.9984933538497378, using 88 bits and 13706 splits
parameters: [ 1  1  5  1  1  1  0  1  1  1  1  1  3 11  5  1 37  2  1] 
 bkg efficiency = -0.9887888729622791, using 35 bits and 111 splits
parameters: [  9   6   7   0   8   3  12   4   6   1   8   4  11   6   5   0 362   4
   3] 
 bkg efficiency = -0.9968071188819921, using 90 bits and 5293 splits
parameters: [  5   2   3  12   4   4   9   3   6   8   1   1   4   5   8   8 316   6
   2] 
 bkg efficiency = -0.9977861537675237, using 83 bits and 16822 splits
parameters: [  4   3  12   3  10   9   6   6   6   9   0   3   8   9   5   1 177   5
   1] 
 bkg efficiency = -0.9986417666375613, using 94 bits and 4802 splits
parameters: [  1   6   7   9   8  11  12   6   3  10  

...

parameters: [  3   1  12  11   0   6   1   0   1   0  11   9   7   8   4   9 183   2
   1] 
 bkg efficiency = -0.9986196801849971, using 83 bits and 549 splits
parameters: [  6   0   1   6   0   5   4   1   1   0   0   0   6   7   1   7 373   6
   1] 
 bkg efficiency = -0.9985481183514114, using 45 bits and 16321 splits
parameters: [  2   0   0   2   1   5   0   1   6   0  11   1   0   7   0   5 407   3
   1] 
 bkg efficiency = -0.9986919600853797, using 41 bits and 2711 splits
parameters: [  6   6   2  12  10  10   1   0   6   3   9   9  10   5   6   8 442   2
   1] 
 bkg efficiency = -0.9987578152244414, using 103 bits and 1326 splits
parameters: [ 1  0  0  0  0  1  0  0  1  0  0  1  0  0  0  0 11  2  1] 
 bkg efficiency = -0.8796384786804821, using 4 bits and 25 splits
parameters: [  1   6  12   0   0   3   0   0   5   0   2   1   0   9   0   0 214   7
   1] 
 bkg efficiency = -0.9958260758133731, using 39 bits and 12051 splits
parameters: [ 1  0  0  1  0  2  0  2  1  0  0  1  0  0 

parameters: [  3   0   0   3   0   1   1   1   1   0   8   0   7   0   2   4 111   4
   1] 
 bkg efficiency = -0.9972696518375187, using 31 bits and 1393 splits
parameters: [  6   0   1   9   9  10   1   6   2   3  11  10   7   1   4   8 463   2
   1] 
 bkg efficiency = -0.9987919436527349, using 88 bits and 1388 splits
parameters: [ 0  0  7  0  0  2  0  1  1  0  0  0  2  0  0  0 11  2  1] 
 bkg efficiency = -0.9323699071038989, using 13 bits and 25 splits
parameters: [  5   6  12   9  11   9   0   5   6   1  11  10   9   1   4   8 352   2
   1] 
 bkg efficiency = -0.9987702293662228, using 107 bits and 1055 splits
parameters: [  4   0   0   0   1   2   0   0   6   0   0   0   1   0   3   0 111   4
   1] 
 bkg efficiency = -0.9891597297555923, using 17 bits and 1294 splits
parameters: [  6   0   0   7   1   8   0   0   1   1  11   2   0   3   0   5 314   2
   1] 
 bkg efficiency = -0.9985977286203682, using 45 bits and 931 splits
parameters: [ 4  1  1  0  8  9  1  1  1  0  1  1  9  0  

/home/llr/cms/hakimi/.local/lib/python3.7/site-packages/pymoo/operators/crossover/simulated_binary_crossover.py:45: RuntimeWarning: invalid value encountered in power
  betaq[mask] = np.power((rand * alpha), (1.0 / (self.eta + 1.0)))[mask]


parameters: [  5   0   0   0   0   4   0   0   1   0   9   1   0   3   0   0 348   7
   1] 
 bkg efficiency = -0.9973664230242134, using 23 bits and 18148 splits
parameters: [  0   0   0   0   1   5   3   1   1   0   3   1   0   0   0   0 493   6
   3] 
 bkg efficiency = -0.9936103313559611, using 15 bits and 15347 splits
parameters: [  5   6  10   9   8  11   0   6   6   3  11   2   9   5   4   5 345   2
   1] 
 bkg efficiency = -0.9987953639299818, using 100 bits and 1035 splits
parameters: [  0   0   0   0   0   2   0   1   1   0   0   0   0   0   0   0 172   4
   1] 
 bkg efficiency = -0.9307728686752681, using 4 bits and 1211 splits
parameters: [  6   6   1  11   1   7   0   0   1   3  10   2   0   1   4   5 314   2
   1] 
 bkg efficiency = -0.9985317470516907, using 58 bits and 939 splits
parameters: [  0   0   0   0   0   2   0   0   1   0   0   1   0   0   0   0 487   2
   2] 
 bkg efficiency = -0.9616796811591819, using 4 bits and 1416 splits
parameters: [  9   5   9  12   3  

parameters: [ 0  0  0  0  0  3  0  1  1  0  0  1  0  0  0  0 11  7  1] 
 bkg efficiency = -0.9860472752350405, using 6 bits and 170 splits
parameters: [  3   2   0   0  10   9   2   0   1   1  10  10   6   0   2   5 101   7
   1] 
 bkg efficiency = -0.9985953420387828, using 61 bits and 7169 splits
parameters: [  4   0   1   6   8   3   1   1   6   3  11   2   9   1   7   0 200   2
   1] 
 bkg efficiency = -0.9984664408130384, using 63 bits and 596 splits
parameters: [  0   0   0   0   0   5   0   1   1   0   3   1   0   0   0   0 493   7
   1] 
 bkg efficiency = -0.993930920802455, using 11 bits and 19666 splits
parameters: [  0   0   0   0   0   1   0   0   1   0   0   1   0   0   0   0 119   3
   1] 
 bkg efficiency = -0.880294781048497, using 3 bits and 504 splits
parameters: [  4   0   0   1   0   6   0   1   1   0   0   1   6   3   0   0 478   2
   1] 
 bkg efficiency = -0.9964011516049897, using 23 bits and 1406 splits
parameters: [  8   0  12   9   0  10   1   5   6   3  11   9

In [42]:
moo_name= '40gen_diag_auc_mut10_3d'

In [43]:
import pickle as pkl
pkl.dump(res, open(savedir + '{}.pkl'.format(moo_name), 'wb'))

In [44]:
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt

%matplotlib notebook

fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, projection='3d', )

# load some test data for demonstration and plot a wireframe
X=res_df['auc']*(-1)
Y=res_df['nbits']
Z=res_df['nsplits']
#X, Y, Z = axes3d.get_test_data(0.1)
ax.scatter(X, Y, Z)
ax.set_xlabel('auc')
ax.set_ylabel('nbits')
ax.set_zlabel('nsplits')
ax.set_xlim(0.99,1)
# rotate the axes and update
for angle in range(0, 360):
    ax.view_init(30, angle)
    plt.draw()
    plt.pause(.001)

<IPython.core.display.Javascript object>

NameError: name 'res_df' is not defined

In [ ]:

%matplotlib notebook

fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, projection='3d', )
if 'auc' in moo_name:
    coeff=(-1,1,1)
    title = 'AUC > 0.8 tpr'
    ax.set_xlim(0.99, 1)
else:
    coeff= (1,1,1)
    title= 'bkg eff at 0.99 signal efficiency'
    i=0
    X,Y,Z=(res.history[i].pop.get("F")*coeff).T
    ax.scatter(X, Y, Z, label= 'gen {}'.format(i+1))
for i, hist in enumerate(res.history):
    if (i+1)%10==0:
        X,Y,Z=(res.history[i].pop.get("F")*coeff).T
        ax.scatter(X, Y, Z, label= 'gen {}'.format(i+1))

#plt.vlines(x=f1, ymin=30, ymax= 170, label = "unquantized")

#ax.set_ylim(0,150)
#ax.set_zlim(0,100)
ax.set_xlabel(title)
ax.set_ylabel('number of bits used')
ax.set_zlabel('number of splits in model')
plt.legend()
#plt.savefig(savedir + '{}.png'.format(moo_name))

In [ ]:
%matplotlib notebook

fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, projection='3d', )
if 'auc' in moo_name:
    coeff=(-1,1,1)
    title = 'AUC > 0.8 tpr'
    ax.set_xlim(0.99, 1)
else:
    coeff= (1,1,1)
    title= 'bkg eff at 0.99 signal efficiency'
res_df=pd.DataFrame(res.history[0].pop.get("F")*coeff)
X=
for i, hist in enumerate(res.history):
    if (i+1)%5==0:
        Plot.add(hist.pop.get("F")*coeff, label = 'gen {}'.format(i+1))
Plot.show()
#plt.vlines(x=f1, ymin=30, ymax= 170, label = "unquantized")
if 'auc' in moo_name:
    plt.xlim(0.99, 1.)
plt.ylim(0,150)

plt.xlabel(title)
plt.ylabel('number of bits used')
plt.legend()
plt.savefig(savedir + '{}.png'.format(moo_name))

In [ ]:
from pymoo.visualization.scatter import Scatter

Plot=Scatter(figsize=(10,10)).add(res.F).show()
#plt.vlines(x=f1, ymin=30, ymax= 85, label = "unquantized")
plt.ylim(30,85)
plt.xlabel('bkg efficiency')
plt.ylabel('number of bits used')
#plt.zlabel('number of splits')
plt.title('pareto front')

In [ ]:
res.history[0].popb.get("F")*(-1,1)

In [ ]:
len(res.history)

In [ ]:
Plot=Scatter(figsize=(10,10))
if 'auc' in moo_name:
    coeff=(-1,1)
    title = 'AUC > 0.8 tpr'
    plt.xlim(0.99, 1)
else:
    coeff= (1,1)
    title= 'bkg eff at 0.99 signal efficiency'
Plot.add(res.history[0].pop.get("F")*coeff, label = 'gen {}'.format(1))
for i, hist in enumerate(res.history):
    if (i+1)%5==0:
        Plot.add(hist.pop.get("F")*coeff, label = 'gen {}'.format(i+1))
Plot.show()
#plt.vlines(x=f1, ymin=30, ymax= 170, label = "unquantized")
if 'auc' in moo_name:
    plt.xlim(0.99, 1.)
plt.ylim(0,150)

plt.xlabel(title)
plt.ylabel('number of bits used')
plt.legend()
plt.savefig(savedir + '{}.png'.format(moo_name))

In [ ]:
a=[100*a for (a,b) in res.F]
b=[int(b) for (a,b) in res.F]

In [ ]:
features

In [ ]:
if bkg == 'QCD':
    feats= ['abseta',
 '1stlay',
 'vee',
 'vpp',
 'vzz',
 'vrr',
 'meanz',
 'sho_lgth',
 'coreshlgth',
 'EoT',
 'ebm1',
 'Emax_4R',
 'rev_ebm1',
 'first_2',
 'Emax_2R',
 'Emax_5']
elif bkg =='PU':
    feats = ['abseta', '1stlay', 'vee', 'vpp', 'vzz', 'vrr',
                 'meanz', 'sho_lgth','core_lgth', 'EoT',
                'firstH_5', 'Emax_4R', 'ebm1', 'Emax_2R', 'firstH_1', 'Emax_5']
elif bkg == 'pions':
    feats = ['abseta',
 '1stlay',
 'vee',
 'vpp',
 'vzz',
 'vrr',
 'meanz',
 'sho_lgth',
    'core_lgth',
 'EoT',
 'Emax_5',
 'ebm1',
 'Emax_4R',
 'hbm',
 'firstH_1',
 'rev_ebm1']

In [ ]:
result= pd.DataFrame(res.X, columns=feats)
result['bkg_efficiency']=a
result['number of bits']=b

result.sort_values(["bkg_efficiency"])
result.to_csv(savedir + '{}_lastgen.csv'.format(moo_name))

In [ ]:
result

In [ ]:
x=xu
qtrain=pd.DataFrame()
qtrain_uni=pd.DataFrame()
qtest=pd.DataFrame()
for i,feat in enumerate(X_train.columns):
    print("feature :", feat, "quantized on {} bits".format(x[i]))
    qtrain[feat]=quantize(X_train[feat], x[i], 'uniform', data[feat].min(), data[feat].max())
    #qtrain_uni[feat]=quantize(X_train[feat], x[i], 'uniform', data[feat].min(), data[feat].max())
    qtest[feat]=quantize(X_test[feat], x[i], 'uniform', data[feat].min(), data[feat].max())

train= xgb.DMatrix(data=X_train,label=y_train['signal'], feature_names=features)
test= xgb.DMatrix(data=X_test,label=y_test['signal'],feature_names=features)

#use gpu if available
gpu=tf.test.is_gpu_available(
    cuda_only=False, min_cuda_compute_capability=None
)
if gpu==True:
    param['tree_method']='gpu_hist'

num_boost_round=200
es_rounds=3

metrics={'logloss','auc'} #last one used for es
# set up cross validation:
#print('beginning cv')
cv_results = xgb.cv(
    param,
    train,
    num_boost_round=num_boost_round,
    seed=42,
    nfold=5,
    metrics=metrics,
    #feval=SoverB,
    early_stopping_rounds=es_rounds,
    shuffle=True,
    verbose_eval=None
)

## we define best score as the best test_auc penalized by the gap between train and test auc
alpha=1 #(penalization)

best_score=(cv_results['test-auc-mean']-alpha*abs(cv_results['test-auc-mean']-cv_results['train-auc-mean'])).max()
boost_rounds = (cv_results['test-auc-mean']-alpha*abs(cv_results['test-auc-mean']-cv_results['train-auc-mean'])).argmax()
#Now train and fit best parameters BDT
eval_result = {}
BDT = xgb.train(
    param,
    train,
    num_boost_round=int(boost_rounds),
    #feval = SoverB,
    evals=[(train, 'Train'),(test, "Test")],
    evals_result = eval_result,
    verbose_eval=None,
)
pred= BDT.predict(test)
fpr, tpr, threshold = roc_curve(y_test['signal'],pred, pos_label=1)
fpr.sort()
tpr.sort()
roc_test=pd.DataFrame({'tpr':tpr,'fpr':fpr, 'threshold':threshold})

#return bkg eff at 0.99 signal efficiency
f1=np.interp(0.99,roc_test['tpr'],roc_test['fpr'])

#f2 = total numer of bits used
f2=int(sum(x))
print('done')

In [ ]:
f1

In [ ]:
perc = 7.6%
uni = 2.9%

In [ ]:
plt.figure(figsize=(10,10))
roc_auc_test = auc(fpr, tpr)
plt.plot(tpr,fpr, label =' AUC = {:2.2%}' .format(roc_auc_test))
plt.yscale('log')
plt.xlim(0.98,1.001)
plt.ylim(3e-2,5e-1)
plt.xlabel('Signal efficiency')
plt.ylabel('Background efficiency')
plt.title('BDT ROC curve (log)')
plt.grid()
plt.legend(loc='upper left')
print(f1)

In [ ]:
Counter((qtrain['cl3d_firstlayer']))

In [ ]:
Counter(y_test['signal'])

In [ ]:
feat=X_train['cl3d_firstlayer']
nbits=4
method='percentile'
fmin=data['cl3d_firstlayer'].min()
fmax=data['cl3d_firstlayer'].max()
print(fmin, fmax)
 #feat is the feature to quantize, nbist the number of bits, method: 'uniform' or 'percentile'
nbins= 2**nbits
if method == 'uniform':
    bins=np.linspace(fmin, fmax, nbins+1)
elif method == 'percentile':
    bins=[np.percentile(feat, n) for n in np.linspace(0,100,nbins+1)]
else :
    print('Error: Invalid method')

print(bins)

np.digitize(feat, bins)

In [ ]:
for i,feat in enumerate(features):
    print(feat)
    plt.figure(figsize=(10,6))
    bins= xu[i]
    plt.hist(qtrain_uni[feat][y_train['signal']==0], bins = bins, label='uniform')
    plt.hist(qtrain[feat][y_train['signal']==0],bins = bins, histtype='step', label = 'percentile)')
    plt.title(feat)
    plt.legend()
    

In [ ]:
qtrain_uni

In [ ]:
for i,feat in enumerate(X_train.columns):
    print(feat,X_train[feat].dtype)

In [ ]:
import pickle as pkl

result={}
for i,item in enumerate(os.listdir(savedir)):
    if "pkl" in item:
        print(i,savedir+item)
        result[i]=pkl.load(open(savedir+item, 'rb'))
        

i=1
feats= ['abseta',
 '1stlay',
 'vee',
 'vpp',
 'vzz',
 'vrr',
 'meanz',
 'sho_lgth',
 'coreshlgth',
 'EoT',
 'ebm1',
 'Emax_4R',
 'rev_ebm1',
 'first_2',
 'Emax_2R',
 'Emax_5']
a=[100*a for (a,b) in result[i].F]
b=[int(b) for (a,b) in result[i].F]
print(os.listdir(savedir)[i])
res_df= pd.DataFrame(result[i].X, columns=feats)
res_df['bkg_efficiency']=a
res_df['number of bits']=b
res_df.sort_values(["bkg_efficiency"])

In [ ]:
res_df.iloc[23]

In [ ]:
feats

In [ ]:
import plotly.graph_objects as go

In [ ]:
dimensions=[]
for ft in feats:
    tmp = dict(range=[0,10],
              #constraintrange = [0,12],
              label= ft,
              values= result[ft])
    dimensions.append(tmp)
    

In [ ]:
result

In [ ]:
fig = go.Figure(data = 
               go.Parcoords(
                    line = dict(color = result['bkg_efficiency']*(-1),
                            colorscale = 'Bluered',
                            showscale =True,
                            cmin = 99,
                            cmax=100
                               ),
                    dimensions = dimensions
                              
                              
                           )
               )
fig.show()
fig.write_image(savedir + '{}_par.png'.format(moo_name))

In [ ]:
os.makedirs(savedir + 'feats', exist_ok=True)
for ft in feats:
    plt.figure(figsize=(10,8))
    plt.scatter(result['bkg_efficiency'].values*(-1), result['number of bits'], c= result[ft])
    #plt.ylim(0,85)
    plt.colorbar()
    plt.xlabel('bkg efficiency')
    plt.ylabel('number of bits used')
    plt.title(ft)
    plt.savefig(savedir+ '/feats/{}.png'.format(ft))
    

In [ ]:
result[['bkg_efficiency', 'number of bits']].values

In [ ]:
plt.scatter(result['bkg_efficiency'].values* (-1), result['number of bits'], c= result[ft])

In [ ]:
model_path= "/grid_mnt/vol_home/llr/cms/hakimi//HGCAL/res/res_pt20_vsPU_featset2/comb/model.model"

In [ ]:
bst= xgb.Booster()
bst.load_model(model_path)

In [ ]:
bst.dump_model('xgb_model.txt', with_stats=True)
# read the contents of the file
with open('xgb_model.txt', 'r') as f:
    txt_model = f.read()
print(txt_model)

In [ ]:
bst.trees_to_dataframe()

In [ ]:

len(bst.trees_to_dataframe()[bst.trees_to_dataframe()['Feature']!='Leaf'])